# Use Plugins with GPT Functions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/edreisMD/plugnplai/blob/main/examples/plugins_step_by_step.ipynb)

The goal of this example is to go through all the steps to add plugins to LLMs
1. Get plugins of certain categories from [plugnplai.com](https://plugnplai.com)
2. Load plugins manifest and specifications
3. Parse specifications and generate a prompt with the descriptions
4. Use [LangChain]() to call the LLM
5. Parse the LLM response, looking for the `[API]` pattern defined on `plugins.prompt`
6. Call the plugin using `plugins.call()`
7. Use LangChain again to ask the LLM a final response using the new data

# Install

In [1]:
pip install -i https://test.pypi.org/simple/ plugnplai==0.0.22a1

Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [2]:
import plugnplai as pl

print(pl.__version__)

0.0.22a1


# Get the plugins

We want to install at maximum three plugins, in order to fit the description on the context length

Lets find one plugin for each category:
1. travel
2. shopping
3. weather

We can use PlugnPlai categories (see [API reference](https://plugnplai.github.io/))

In [3]:
import plugnplai as pl
# Get working plugins - only tested plugins (in progress)
allUrls = pl.get_plugins()

# Lets pick Trip, Klarna and Speak
urls = [plugin for plugin in allUrls if 'klarna' in plugin]

print(f'Our chosen Plugins: {urls}')

Our chosen Plugins: ['https://klarna.com']


# Load and activate the plugins

In [4]:
from plugnplai import Plugins

plugins = Plugins.install_and_activate(urls)

## Show the function object

In [5]:
plugins.functions

[{'name': 'KlarnaProducts__opid__productsUsingGET',
  'description': 'Assistant uses the Klarna plugin to get relevant product suggestions for any shopping or product discovery purpose. Assistant will reply with the following 3 paragraphs 1) Search Results 2) Product Comparison of the Search Results 3) Followup Questions. The first paragraph contains a list of the products with their attributes listed clearly and concisely as bullet points under the product, together with a link to the product and an explanation. Links will always be returned and should be shown to the user. The second paragraph compares the results returned in a summary sentence starting with "In summary". Assistant comparisons consider only the most important features of the products that will help them fit the users request, and each product mention is brief, short and concise. In the third paragraph assistant always asks helpful follow-up questions and end with a question mark. When assistant is asking a follow-up 

In [6]:
plugins.functions[0]["description"] = "Assistant uses the Klarna plugin to get relevant product suggestions for any shopping or product discovery purpose."

## Lets look at the length of the prompt

Get the number of tokens of the prompt by just calling 'plugins.tokens'

In [7]:
print(plugins.func_tokens)

573


## Call the LLM with functions

In [8]:
# You will need to first define your API key
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

#### Uncomment or modify the message to test different plugins

In [9]:
# Test Klarna Plugin
HUMAN_MESSAGE = "I want to buy a rolling stones t-shirt"

# Test Trip Plugin
# HUMAN_MESSAGE = "I need a hotel in Paris between Dec.3-8"

# Test Speak Plugin
# HUMAN_MESSAGE = "How to say I love you in Portuguese?"

#### Call LLM

In [10]:
import openai
import json

def use_call_api(response_message):
    
    function_name = response_message["function_call"]["name"]
    split_name = function_name.split("__opid__")
    plugin_name = split_name[0]
    operation_id = split_name[1]
    parameters = response_message["function_call"]["arguments"]
    
    r = plugins.call_api(plugin_name = plugin_name,
                        operation_id = operation_id,
                        parameters = parameters
                        )

    api_response = r.json()
    return r.json()


def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": HUMAN_MESSAGE}]
    
    # Use 'plugins.functions' on the gpt call 
    functions = plugins.functions
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        functions=functions,
        # auto is default, but here we will force GPT to call the function
        function_call= {"name": "KlarnaProducts__opid__productsUsingGET"},
    )
    
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        # now use plugins.call_api defined on the use_call_api above
        function_response = use_call_api(response_message)

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": response_message["function_call"]["name"],
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response


print(run_conversation())

InvalidRequestError: Invalid schema for function 'KlarnaProducts__opid__productsUsingGET': schema must be a JSON Schema of 'type: "object"', got 'type: "None"'.